In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import sys
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import keras_tuner as kt
from sklearn.metrics import classification_report
import shutil
from os import path
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator

# path variables to access files
root = os.getcwd()

### Load dataframe

In [2]:
main_data = pd.read_csv("data_labels_mainData.csv")
extra_data = pd.read_csv("data_labels_extraData.csv")

### Get data from splitted folder

In [3]:
# method get data for multiclass task (task 2)
def celltype_classify_data_all(class_list):    
    images = list()
    labels = list()
    
    for i, label in enumerate(class_list):
        # get image directory
        img_dir = os.path.join(root, "multiclass_task", f"{label}")
        
        for img in os.listdir(img_dir):
            img = cv2.imread(os.path.join(img_dir, img))
            # resize to 0-1 for faster computation
            resized = img / 255
            images.append(resized)
            labels.append(i)
        
    return (images, labels)

all_class = ["epithelial", "fibroblast", "inflammatory", "others"]

# method get data for multiclass task (task 2)
def celltype_classify_data(class_list, mode):    
    images = list()
    labels = list()
    
    for i, label in enumerate(class_list):
        # get image directory
        img_dir = os.path.join(root, "split3-multi-task", f"{mode}", f"{label}")
        
        for img in os.listdir(img_dir):
            img = cv2.imread(os.path.join(img_dir, img))
            # resize to 0-1 for faster computation
            resized = img / 255
            images.append(resized)
            labels.append(i)
        
    return (images, labels)

### Classification report method

In [4]:
def multiclass_classification_report(y_test, prediction, print_out=True):
    """
        Method to generate sklearn classification report with CNN multiclass output
    """
    
    encoded_pred = list()
    # convert each CNN output (sparse categorial) to class
    for pred in prediction:
        encoded_pred.append(np.argmax(pred))

    encoded_pred = np.array(encoded_pred)
    if print_out:
        print(classification_report(y_test, encoded_pred))
    
    return classification_report(y_test, encoded_pred, output_dict = True)

### Structure of sub-models

In [5]:
def get_multiclass_model():
    """
        Structure of model classifying 4 classes
    """
    
    model = tf.keras.Sequential()
    # First convo-pooling
    # Convolutional layers (filter the image with a kernel)
    model.add(tf.keras.layers.Conv2D(64, (3, 3), strides=1,activation="relu", input_shape=[27, 27, 3]))
    # Max-pooling layers (reduce the size of the image by choosing max pixel at certain area)
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=1))
    
    # Second convo-pooling
    # Convolutional layers (filter the image with a kernel)
    model.add(tf.keras.layers.Conv2D(64, (3, 3),strides=1, activation="relu"))
    # Max-pooling layers (reduce the size of the image by choosing max pixel at certain area)
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2),strides=1))
    
    # Flatten input
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation="relu"))
    model.add(tf.keras.layers.Dense(128, activation="relu"))
    model.add(tf.keras.layers.Dense(32, activation="relu"))
    # Output layer
    model.add(tf.keras.layers.Dense(4, activation="softmax"))
    # Compile model 
    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    
    return model

def get_subclass_model():
    """
        Structure of model classifying 3 classes
    """
    
    model = tf.keras.Sequential()
    # First convo-pooling
    # Convolutional layers (filter the image with a kernel)
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu", input_shape=[27, 27, 3]))
    # Max-pooling layers (reduce the size of the image by choosing max pixel at certain area)
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
    # Flatten input
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation="relu"))
    model.add(tf.keras.layers.Dense(128, activation="relu"))
    model.add(tf.keras.layers.Dense(32, activation="relu"))
    # Output layer
    model.add(tf.keras.layers.Dense(3, activation="softmax"))
    # Compile model 
    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    
    return model

### Methods to train and save submodels

In [6]:
submodel_path = os.path.join(root, "multiclass_submodels")   
def three_class_submodel(file_path, aug_train = None, aug_val = None):
    # path to model trained with 3 classes
    subclass_path = path.join(file_path, "subclass.h5")
    subclass_model = None
    if not os.path.isfile(subclass_path) or keras.models.load_model(subclass_path) == None:
        print("---Sub model training---")
        # train subclass that fit with 3 types of images
        subclass_model = get_subclass_model()
        
        if aug_train is not None and aug_val is not None:
            print("Train with augmented data")
            subclass_model.fit(aug_train, epochs=50, validation_data=aug_val)
        else:
            print("Train with non-augmented data")
            subclass_model.fit(subx_train, suby_train, epochs=40, validation_data=(subx_test, suby_test))
            
        subclass_model.save(subclass_path)
        return subclass_model
    else:
        subclass_model = keras.models.load_model(subclass_path)
        print("Model trained with 3 classes loaded")
        return subclass_model
    
def all_class_submodel(file_path, aug_train = None, aug_val = None):
    allclass_path = path.join(file_path, "allclass.h5")
    allclass_model = None
    if not os.path.isfile(allclass_path) or keras.models.load_model(allclass_path) == None:
        print("---Sub model training---")
        # train subclass that fit with 4 types of images
        allclass_model = get_multiclass_model()
        
        if aug_train is not None and aug_val is not None:
            print("Train with augmented data")
            allclass_model.fit(aug_train, epochs=50, validation_data=aug_val)
        else:
            print("Train with non-augmented data")
            allclass_model.fit(x_train, y_train, epochs=15, validation_data=(x_val, y_val))
            
        allclass_model.save(allclass_path)
        return allclass_model
    else:
        allclass_model = keras.models.load_model(allclass_path)
        print("Model trained with 4 classes loaded")
        return allclass_model

### Meta-learner

In [7]:
def get_transfer_model():
    model = tf.keras.Sequential()
    # Flatten input
    model.add(tf.keras.layers.Flatten())
    # Hidden layers
    model.add(tf.keras.layers.Dense(256, activation="relu"))
    model.add(tf.keras.layers.Dense(128, activation="softmax"))
    model.add(tf.keras.layers.Dense(32, activation="relu"))
    model.add(tf.keras.layers.Dense(3, activation="sigmoid"))
    # Output layer
    model.add(tf.keras.layers.Dense(4, activation="softmax"))
    # Compile model
    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

### Get augmentated data for training

In [9]:
# load images from folders
all_class = ["epithelial", "fibroblast", "inflammatory", "others"]
cell_img, cell_label = celltype_classify_data_all(all_class)
cell_img, cell_label = np.array(cell_img), np.array(cell_label)

sub_multiclass = ["epithelial", "fibroblast", "inflammatory"]
sub_img, sub_label = celltype_classify_data_all(sub_multiclass)

# # train and validate data for 3 classes
# subx_train, subx_test, suby_train, suby_test = train_test_split(
#     np.array(sub_img), np.array(sub_label), test_size=0.2
# )

# # train and validate data for 4 classes
# x_train_val, x_test, y_train_val, y_test = train_test_split(
#     cell_img, cell_label, test_size=0.2
# )
# x_train, x_val, y_train, y_val = train_test_split(
#     x_train_val, y_train_val, test_size=0.25
# )

x_train, y_train = celltype_classify_data(all_class, "train")
x_train, y_train = np.array(x_train), np.array(y_train)

x_val, y_val = celltype_classify_data(all_class, "val")
x_val, y_val = np.array(x_val), np.array(y_val)

x_test, y_test = celltype_classify_data(all_class, "test")
x_test, y_test = np.array(x_test), np.array(y_test)

sub_multiclass = ["epithelial", "fibroblast", "inflammatory"]
subx_train, suby_train = celltype_classify_data(sub_multiclass, "train")
subx_train, suby_train = np.array(subx_train), np.array(suby_train)
subx_test, suby_test = celltype_classify_data(sub_multiclass, "val")
subx_test, suby_test = np.array(subx_test), np.array(suby_test)


# get augmentation for extra training data
extra_datagen = ImageDataGenerator(
#     width_shift_range=0.3,
#     height_shift_range=0.3,
    rotation_range=60,
    horizontal_flip=True,
    vertical_flip=True,

)

# fit augmentation with all images from all 4 classes
extra_datagen.fit(cell_img)

# get augmentations for unlabeled data
# shift_aug = ImageDataGenerator(
#     width_shift_range=0.3,
#     height_shift_range=0.3,
# )
rotation_aug = ImageDataGenerator(
    rotation_range=60
)
flip_aug = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
)
aug_list = [rotation_aug, flip_aug]

# git aug_list with images
for aug in aug_list:
    aug.fit(cell_img)

### Helper function 

In [10]:
def get_extra_stacked(x, submodels, datagen):
    """Generate stacked output from submodels with augmented x for meta learner"""
    stacked = None
    # augmentate x before predicting with submodels
    pred_gen = datagen.flow(x, shuffle=False)
    for submodel in submodels:
        if stacked is None:
            stacked = submodel.predict_generator(pred_gen)
        else:
            # augmentate x before predicting with submodels
            new_pred = submodel.predict_generator(pred_gen)
            stacked = np.concatenate((stacked, new_pred), axis=1)
    return stacked

def mix_match_pred(x, aug_list, sub_models, meta_learner):
#     get prediction of mix match model
    pred_arr = None
#     Find the sum of all predictions from each augmentation test set
    for aug in aug_list:
        # get stacked output from submodels to let meta_learner predict 
        stacked_x = get_extra_stacked(x, sub_models, aug)
        # predict with metta_learner
        pred = meta_learner.predict(stacked_x)
        
        if pred_arr is None:
            pred_arr = pred
        else:
            pred_arr = np.add(pred_arr, pred)
#     Find the average of prediction      
    pred_arr = np.true_divide(pred_arr, len(aug_list))
    return pred_arr

### 

In [11]:
extramodel_path = os.path.join(root, "extra_submodels")
# data for 3 classes submodels
sub_train = extra_datagen.flow(subx_train, suby_train)
sub_val = extra_datagen.flow(subx_test, suby_test)

# data for 4 classes submodels 
all_train = extra_datagen.flow(x_train, y_train)
all_val = extra_datagen.flow(x_val, y_val)

# train submodels with augmented x
subclass_extra = three_class_submodel(extramodel_path, aug_train=sub_train, aug_val=sub_val)
allclass_extra = all_class_submodel(extramodel_path, aug_train=all_train, aug_val=all_val)
extra_submodels = [subclass_extra, allclass_extra]

# get stacked x for meta-learner
stacked_x_train = get_extra_stacked(x_train,extra_submodels, extra_datagen)
stacked_x_val = get_extra_stacked(x_val,extra_submodels, extra_datagen)

extra_model = get_transfer_model()
# fit model with stacked data from submodels
extra_model.fit(stacked_x_train, y_train, epochs=30, validation_data=(stacked_x_val, y_val))

---Sub model training---
Metal device set to: Apple M1 Pro
Train with augmented data
Epoch 1/50


2022-05-12 18:57:00.317794: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-12 18:57:00.317973: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-05-12 18:57:00.450571: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-05-12 18:57:00.593590: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


154/160 [===========================>..] - ETA: 0s - loss: 0.7411 - accuracy: 0.6755

2022-05-12 18:57:02.192267: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


160/160 [==============================] - 2s 11ms/step - loss: 0.7364 - accuracy: 0.6791 - val_loss: 0.6293 - val_accuracy: 0.7324
Epoch 2/50
160/160 [==============================] - 1s 9ms/step - loss: 0.5746 - accuracy: 0.7692 - val_loss: 0.4969 - val_accuracy: 0.8141
Epoch 3/50
160/160 [==============================] - 2s 9ms/step - loss: 0.5337 - accuracy: 0.7851 - val_loss: 0.5364 - val_accuracy: 0.7853
Epoch 4/50
160/160 [==============================] - 1s 9ms/step - loss: 0.5147 - accuracy: 0.7929 - val_loss: 0.5000 - val_accuracy: 0.8071
Epoch 5/50
160/160 [==============================] - 2s 10ms/step - loss: 0.4977 - accuracy: 0.8002 - val_loss: 0.4919 - val_accuracy: 0.8171
Epoch 6/50
160/160 [==============================] - 1s 9ms/step - loss: 0.4921 - accuracy: 0.8054 - val_loss: 0.4768 - val_accuracy: 0.8153
Epoch 7/50
160/160 [==============================] - 2s 10ms/step - loss: 0.4923 - accuracy: 0.8053 - val_loss: 0.6146 - val_accuracy: 0.7459
Epoch 8/50
160

...

Epoch 1/30
 10/186 [>.............................] - ETA: 1s - loss: 1.5176 - accuracy: 0.2375 

2022-05-12 19:00:37.234203: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


186/186 [==============================] - ETA: 0s - loss: 1.3406 - accuracy: 0.2876

2022-05-12 19:00:38.540740: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


186/186 [==============================] - 2s 8ms/step - loss: 1.3406 - accuracy: 0.2876 - val_loss: 1.1660 - val_accuracy: 0.5873
Epoch 2/30
186/186 [==============================] - 1s 6ms/step - loss: 1.0396 - accuracy: 0.7483 - val_loss: 0.9696 - val_accuracy: 0.7344
Epoch 3/30
186/186 [==============================] - 1s 6ms/step - loss: 0.8917 - accuracy: 0.7570 - val_loss: 0.8749 - val_accuracy: 0.7319
Epoch 4/30
186/186 [==============================] - 1s 6ms/step - loss: 0.8263 - accuracy: 0.7508 - val_loss: 0.8310 - val_accuracy: 0.7339
Epoch 5/30
186/186 [==============================] - 1s 6ms/step - loss: 0.7848 - accuracy: 0.7542 - val_loss: 0.8077 - val_accuracy: 0.7304
Epoch 6/30
186/186 [==============================] - 1s 6ms/step - loss: 0.7539 - accuracy: 0.7520 - val_loss: 0.7774 - val_accuracy: 0.7314
Epoch 7/30
186/186 [==============================] - 1s 6ms/step - loss: 0.7268 - accuracy: 0.7532 - val_loss: 0.7551 - val_accuracy: 0.7294
Epoch 8/30
186/18

In [12]:
# predict with mix match (avg of multiple augmentations)
meta_pred = mix_match_pred(x_test, aug_list, extra_submodels, extra_model)

multiclass_classification_report(y_test,meta_pred);

/var/folders/bc/1x56pk_j3glgqwsdp02fbltm0000gn/T/ipykernel_68356/2772422593.py:8: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  stacked = submodel.predict_generator(pred_gen)
/var/folders/bc/1x56pk_j3glgqwsdp02fbltm0000gn/T/ipykernel_68356/2772422593.py:11: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  new_pred = submodel.predict_generator(pred_gen)
2022-05-12 19:01:28.838864: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


              precision    recall  f1-score   support

           0       0.87      0.93      0.90       817
           1       0.79      0.73      0.76       379
           2       0.73      0.83      0.77       510
           3       0.72      0.49      0.58       278

    accuracy                           0.80      1984
   macro avg       0.78      0.74      0.75      1984
weighted avg       0.80      0.80      0.79      1984



### Get extra dataset

In [13]:
def load_extra_epi():   
    """Load cancerous/epithelial in the extra dataset"""
    images = list()
    labels = list()
    
    # get image directory
    img_dir = os.path.join(root, "extra", "1")

    for img in os.listdir(img_dir):
        img = cv2.imread(os.path.join(img_dir, img))
        # resize to 0-1 for faster computation
        resized = img / 255
        images.append(resized)
        
        labels.append(0)
    return (images, labels)

def load_unlabeled():
    """Load unlabeled data from the extra dataset"""
    images = list()
    
    # get image directory
    img_dir = os.path.join(root, "extra", "0")

    for img in os.listdir(img_dir):
        img = cv2.imread(os.path.join(img_dir, img))
        # resize to 0-1 for faster computation
        resized = img / 255
        images.append(resized)
    return images

### Load extra data

In [14]:
epi_train, epi_label = load_extra_epi()
epi_train, epi_label = np.array(epi_train), np.array(epi_label)

unlabeled = load_unlabeled()
unlabeled = np.array(unlabeled)

extra_model.save(path.join(root, "extra_model.h5"))

print(epi_label.shape)

(2990,)


### Load model

In [15]:
extra_model = keras.models.load_model(path.join(root, "extra_model.h5"))

### Generate additional train data from extra dataset

In [16]:
def encode_labels(pred_labels):
    encoded_list = list()
    # transform pseudo label to consists of integers
    for i, pred in enumerate(pseudo_labels):
        encoded = np.argmax(pred)
        encoded_list.append(encoded)
        
    return np.array(encoded_list)

# predict unlabeled data
pseudo_labels = mix_match_pred(unlabeled, aug_list, extra_submodels, extra_model)
pseudo_labels = encode_labels(pseudo_labels)

/var/folders/bc/1x56pk_j3glgqwsdp02fbltm0000gn/T/ipykernel_68356/2772422593.py:8: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  stacked = submodel.predict_generator(pred_gen)
/var/folders/bc/1x56pk_j3glgqwsdp02fbltm0000gn/T/ipykernel_68356/2772422593.py:11: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  new_pred = submodel.predict_generator(pred_gen)
2022-05-12 19:01:33.273588: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


### Train extra data in batches

In [17]:
import warnings
# mute warning 
warnings.filterwarnings('ignore')

batches = 10
extra_x_batch = np.array_split(unlabeled, batches)
extra_y_batch = np.array_split(pseudo_labels, batches)

# load extra model from last save point
extra_model = keras.models.load_model(path.join(root, "extra_model.h5"))

for i in range(batches):
    # compute accuracy with test set
    old_pred = mix_match_pred(x_test, aug_list, extra_submodels, extra_model)
    old_accuracy = multiclass_classification_report(y_test,old_pred, print_out=False)["accuracy"]
    
    # generate stacked x for train
    stacked_batched = get_extra_stacked(extra_x_batch[i], extra_submodels, extra_datagen)
    # fit with augmented batch
    extra_model.fit(stacked_batched, extra_y_batch[i], epochs=10, validation_data=(stacked_x_val, y_val), verbose=0)
    
    # compute new accuracy with test set
    new_pred = mix_match_pred(x_test, aug_list, extra_submodels, extra_model)
    new_accuracy = multiclass_classification_report(y_test,new_pred, print_out=False)["accuracy"]
    
    # save model if newly trained batch increase accuracy by at least 0.001
    if (new_accuracy - old_accuracy) > 0.001:
        print("Update model with newly trained data")
        extra_model.save(path.join(root, "extra_model.h5"))
    else:
        print("Reload from most recent save")
        # reload from old model/checkpoint if new batch decrease accuracy
        extra_model = keras.models.load_model(path.join(root, "extra_model.h5"))
    
# compute new model stats
updated_pred = mix_match_pred(x_test, aug_list, extra_submodels, extra_model)
multiclass_classification_report(y_test,updated_pred);

2022-05-12 19:01:35.857414: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-12 19:01:36.772772: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-12 19:01:37.055507: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Reload from most recent save


2022-05-12 19:01:41.863148: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-12 19:01:42.751742: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-12 19:01:43.017069: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Reload from most recent save


2022-05-12 19:01:47.776086: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-12 19:01:48.658806: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-12 19:01:48.941488: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

Reload from most recent save


2022-05-12 19:03:59.585466: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-12 19:04:00.514549: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-12 19:04:00.834668: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Reload from most recent save


2022-05-12 19:04:06.086633: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


              precision    recall  f1-score   support

           0       0.87      0.92      0.90       817
           1       0.77      0.72      0.74       379
           2       0.72      0.83      0.77       510
           3       0.69      0.45      0.55       278

    accuracy                           0.79      1984
   macro avg       0.76      0.73      0.74      1984
weighted avg       0.79      0.79      0.79      1984

